In [3]:
import pandas as pd

# 读取三个CSV文件
df1 = pd.read_csv("/data4/tongshuo/dataset/aptos2019/train_1.csv")
df2 = pd.read_csv("/data4/tongshuo/dataset/aptos2019/test.csv")
df3 = pd.read_csv("/data4/tongshuo/dataset/aptos2019/valid.csv")

# 合并这三个DataFrame
merged_df = pd.concat([df1, df2, df3], ignore_index=True)

# 将合并后的数据输出为一个新的CSV文件
merged_df.to_csv('/data4/tongshuo/dataset/aptos2019/ten_fold/all.csv', index=False)
print("CSV files merged successfully!")


CSV files merged successfully!


In [7]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold

# 读取原始数据
df = pd.read_csv('/data4/tongshuo/dataset/aptos2019/ten_fold/all.csv')

# 确保目标列是 'diagnosis'
target_col = 'diagnosis'

# 初始化 StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# 保存每个折的数据
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df[target_col])):
    # 分别获取训练集和验证集
    fold_df = df.iloc[val_idx]
    
    # 输出为CSV文件
    fold_df.to_csv(f'/data4/tongshuo/dataset/aptos2019/ten_fold/fold_{fold}.csv', index=True)

print("Data has been split into 10 folds and saved as CSV files.")


Data has been split into 10 folds and saved as CSV files.


### balance data

In [4]:
import pandas as pd
import numpy as np

# Load your dataset (assuming it's in a CSV file)
df = pd.read_csv('/data4/tongshuo/dataset/aptos2019/ten_fold_balanced/all.csv')

# Step 1: Filter the dataset by diagnosis
diagnosis_0 = df[df['diagnosis'] == 0]
diagnosis_1 = df[df['diagnosis'] == 1]
diagnosis_2 = df[df['diagnosis'] == 2]
diagnosis_3 = df[df['diagnosis'] == 3]
diagnosis_4 = df[df['diagnosis'] == 4]

# Step 2: Randomly select 600 samples from diagnosis 0
diagnosis_0_selected = diagnosis_0.sample(n=600, random_state=42)

# Step 3: Select all samples from diagnosis 1, 3, and 4
diagnosis_1_selected = diagnosis_1
diagnosis_3_selected = diagnosis_3
diagnosis_4_selected = diagnosis_4

# Step 4: Randomly select 300 samples from diagnosis 2
diagnosis_2_selected = diagnosis_2.sample(n=300, random_state=42)

# Step 5: Combine the selected samples
balanced_df = pd.concat([diagnosis_0_selected, diagnosis_1_selected, diagnosis_2_selected, diagnosis_3_selected, diagnosis_4_selected])

# Optionally, shuffle the final dataset if you want random ordering
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# The resulting balanced dataset
print(balanced_df.head())  # Display the first few rows of the balanced dataset
print(len(balanced_df))
# Save the balanced dataset to a new CSV file
balanced_df.to_csv("/data4/tongshuo/dataset/aptos2019/ten_fold_balanced/balanced_all.csv", index=False)


        id_code  diagnosis
0  2700754f71e9          2
1  2608e1dac5b1          2
2  0cb14014117d          3
3  374535e0adb8          4
4  83038ca49b6d          0
1758


In [5]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold

# 读取原始数据
df = pd.read_csv('/data4/tongshuo/dataset/aptos2019/ten_fold_balanced/balanced_all.csv')
# 确保目标列是 'diagnosis'
target_col = 'diagnosis'

# 初始化 StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# 保存每个折的数据
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df[target_col])):
    # 分别获取训练集和验证集
    fold_df = df.iloc[val_idx]
    
    # 输出为CSV文件
    fold_df.to_csv(f'/data4/tongshuo/dataset/aptos2019/ten_fold_balanced/fold_{fold}.csv', index=True)

print("Data has been split into 10 folds and saved as CSV files.")


Data has been split into 10 folds and saved as CSV files.


In [2]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold

# 读取原始数据
df = pd.read_excel("/data3/tongshuo/dataset/image/SICAPv2/ten_fold/all.xlsx")

# 确保目标列是 'diagnosis'
target_col = 'label'

# 初始化 StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# 保存每个折的数据
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df[target_col])):
    # 分别获取训练集和验证集
    fold_df = df.iloc[val_idx]
    
    # 输出为CSV文件
    fold_df.to_csv(f'/data3/tongshuo/dataset/image/SICAPv2/ten_fold/fold_{fold}.csv', index=True)

print("Data has been split into 10 folds and saved as CSV files.")


Data has been split into 10 folds and saved as CSV files.


In [10]:
import torch
import torch.utils.data as data_utils
import numpy as np
import os
from PIL import Image, ImageDraw
from PIL import Image
import copy
import pandas as pd
import csv
import random
from typing import Tuple, Dict
from torch import Tensor
import torchvision.transforms as transforms
from torchvision.transforms import InterpolationMode
import pickle

class APTOSDataset(data_utils.Dataset):

    def __init__(self, patch_level, img_root, dataset, transform=None, fold=3):
        self.data_list = []
        # self.items = []
        self.transform = transform
        self.patch_level = patch_level
        if dataset == 'train':
            data_num = [i for i in range(10) if i != fold]
        elif dataset == 'valid':
            data_num = [fold]
        print("Use the data fold: {}".format(data_num))
        for i in data_num:
            #f = open('data_split/dr/fold_{}.csv'.format(i), "r")
            f = open('/data4/tongshuo/dataset/aptos2019/ten_fold/fold_{}.csv'.format(i), "r")
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.data_list.append(row[1:])

        self.label_list = [int(x[-1]) for x in self.data_list] #提出了所有折中的label
        # 提取出0, 1, 2
        #self.label_list = [x for x in self.label_list if x in {0, 1, 2}]

        if self.patch_level == 1:
            self.patch_class = self.get_token_class(fold)

        self.label_num = [0, 0, 0, 0, 0]
        for each in self.label_list:
            self.label_num[each] += 1
        print(self.label_num) #[23229, 2199, 4763, 786, 638]
        print(len(self.data_list)) #31615

    def __getitem__(self, idx):
        item = copy.deepcopy(self.data_list[idx])
        print(item)
        img = item[0]
        label = int(item[1])
        img_path = '/data4/tongshuo/dataset/aptos2019/images/' + img + '.png'
        # label = int(item[-1])
        img = Image.open(img_path).convert('RGB')

        if self.transform:
            img = self.transform(img)

        return img, label

    def __len__(self):
        return len(self.data_list)


if __name__ == '__main__':
    tran = transforms.Compose([
            transforms.Resize((256, 256), interpolation=InterpolationMode.BILINEAR),
            transforms.RandomResizedCrop(224, scale=(0.08, 1.)),
            transforms.RandomApply([
                transforms.ColorJitter(0.4, 0.4, 0.2, 0.1)  # not strengthened
            ], p=0.8),
            transforms.RandomGrayscale(p=0.2),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
    train_data = APTOSDataset(None,None,'train',tran,0)
    # loader = torch.utils.data.DataLoader(train_data, batch_size=2, shuffle=True, num_workers=4, drop_last=True)
    loader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True, drop_last=True)
    for i, (a, b) in enumerate(loader):
        print('i:',i)
        print('###',a.shape, b.shape)
        # 判断失败案例（预测错误的样本）
        # if predicted[i] != label[i]:
        #     # 记录失败的索引、真实标签和预测标签
        #     failed_cases.append((batch_idx * len(label) + i, label[i].item(), predicted[i].item()))
        #     # 记录失败案例的图像路径
        #     img_path = train_loader.dataset.data_list[batch_idx * len(label) + i][0]  # 获取图像的文件名
        #     failed_images.append(img_path)
        break

Use the data fold: [1, 2, 3, 4, 5, 6, 7, 8, 9]
[1625, 333, 899, 173, 265]
3295
['a0d04a19cf40', '2']
['27bab1432f61', '2']
['b22354b5f94b', '1']
['a93f1ea3ff4a', '2']
['cf603a9ef2d5', '1']
['ab1c20a94f3f', '4']
['79d44db3da2d', '4']
['0d0b8fc9ab5c', '0']
i: 0
### torch.Size([8, 3, 224, 224]) torch.Size([8])


In [5]:
import torch
import torch.utils.data as data_utils
import numpy as np
import os
from PIL import Image, ImageDraw
from PIL import Image
import copy
import pandas as pd
import csv
import random
from typing import Tuple, Dict
from torch import Tensor
import torchvision.transforms as transforms
from torchvision.transforms import InterpolationMode
import pickle

class SICAPv2Dataset(data_utils.Dataset):

    def __init__(self, patch_level, img_root, dataset, transform=None, fold=3):
        self.data_list = []
        # self.items = []
        self.transform = transform
        self.patch_level = patch_level
        if dataset == 'train':
            data_num = [i for i in range(10) if i != fold]
        elif dataset == 'valid':
            data_num = [fold]
        print("Use the data fold: {}".format(data_num))
        for i in data_num:
            f = open('/data3/tongshuo/dataset/image/SICAPv2/ten_fold/fold_{}.csv'.format(i), "r")
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.data_list.append(row[1:])

        self.label_list = [int(x[-1]) for x in self.data_list] #提出了所有折中的label
        # 提取出0, 1, 2
        #self.label_list = [x for x in self.label_list if x in {0, 1, 2}]

        if self.patch_level == 1:
            self.patch_class = self.get_token_class(fold)

        self.label_num = [0, 0, 0, 0]
        for each in self.label_list:
            self.label_num[each] += 1
        print(self.label_num) #[23229, 2199, 4763, 786, 638]
        print(len(self.data_list)) #31615

    def __getitem__(self, idx):
        item = copy.deepcopy(self.data_list[idx])
        print(item)
        img = item[0]
        label = int(item[1])
        img_path = '/data3/tongshuo/dataset/image/SICAPv2/images/' + img
        # label = int(item[-1])
        img = Image.open(img_path).convert('RGB')

        if self.transform:
            img = self.transform(img)

        return img, label

    def __len__(self):
        return len(self.data_list)


if __name__ == '__main__':
    tran = transforms.Compose([
            transforms.Resize((256, 256), interpolation=InterpolationMode.BILINEAR),
            transforms.RandomResizedCrop(224, scale=(0.08, 1.)),
            transforms.RandomApply([
                transforms.ColorJitter(0.4, 0.4, 0.2, 0.1)  # not strengthened
            ], p=0.8),
            transforms.RandomGrayscale(p=0.2),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
    train_data = SICAPv2Dataset(None,None,'train',tran,0)
    # loader = torch.utils.data.DataLoader(train_data, batch_size=2, shuffle=True, num_workers=4, drop_last=True)
    loader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True, drop_last=True)
    for i, (a, b) in enumerate(loader):
        print('i:',i)
        print('###',a.shape, b.shape)
        break

Use the data fold: [1, 2, 3, 4, 5, 6, 7, 8, 9]
[3975, 2000, 4044, 853]
10872
['17B0031079_Block_Region_6_12_16_xini_30193_yini_65523.jpg', '2']
['18B0004346A_Block_Region_1_10_5_xini_28790_yini_68945.jpg', '2']
['18B0005718B_Block_Region_8_5_8_xini_30489_yini_15722.jpg', '2']
['17B0023909_Block_Region_5_10_23_xini_28384_yini_16198.jpg', '2']
['18B0003032A_Block_Region_2_21_30_xini_36859_yini_22888.jpg', '2']
['17B0011996_Block_Region_5_10_11_xini_17887_yini_63887.jpg', '1']
['16B0006668_Block_Region_3_25_7_xini_61646_yini_93244.jpg', '3']
['16B0027040_Block_Region_10_21_14_xini_42926_yini_37492.jpg', '0']
i: 0
### torch.Size([8, 3, 224, 224]) torch.Size([8])


### 12081: 4417, 2222, 4494, 948 -> 2500, 2222, 2500, 948

In [2]:
import pandas as pd
import numpy as np

# Load your dataset (assuming it's in a CSV file)
df = pd.read_excel("/data3/tongshuo/dataset/image/SICAPv2/ten_fold_balanced/all.xlsx")

# Step 1: Filter the dataset by diagnosis
diagnosis_0 = df[df['label'] == 0]
diagnosis_1 = df[df['label'] == 1]
diagnosis_2 = df[df['label'] == 2]
diagnosis_3 = df[df['label'] == 3]

# Step 2: Randomly select 600 samples from diagnosis 0
diagnosis_0_selected = diagnosis_0.sample(n=2500, random_state=42)

# Step 3: Select all samples from diagnosis 1, 3, and 4
diagnosis_1_selected = diagnosis_1
diagnosis_3_selected = diagnosis_3

# Step 4: Randomly select 300 samples from diagnosis 2
diagnosis_2_selected = diagnosis_2.sample(n=2500, random_state=42)

# Step 5: Combine the selected samples
balanced_df = pd.concat([diagnosis_0_selected, diagnosis_1_selected, diagnosis_2_selected, diagnosis_3_selected])

# Optionally, shuffle the final dataset if you want random ordering
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# The resulting balanced dataset
print(balanced_df.head())  # Display the first few rows of the balanced dataset
print(len(balanced_df))
# Save the balanced dataset to a new CSV file
balanced_df.to_csv("/data3/tongshuo/dataset/image/SICAPv2/ten_fold_balanced/balanced_all.csv", index=False)


                                          image_name  label
0  17B0031877_Block_Region_4_14_14_xini_21274_yin...      2
1  18B0001159D_Block_Region_3_7_28_xini_37520_yin...      2
2  17B0017506_Block_Region_1_2_3_xini_8727_yini_1...      1
3  17B0023909_Block_Region_5_13_17_xini_22240_yin...      2
4  18B0004349G_Block_Region_15_0_1_xini_24248_yin...      2
8170


In [4]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold

# 读取原始数据
df = pd.read_csv('/data3/tongshuo/dataset/image/SICAPv2/ten_fold_balanced/balanced_all.csv')
# 确保目标列是 'diagnosis'
target_col = 'label'

# 初始化 StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# 保存每个折的数据
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df[target_col])):
    # 分别获取训练集和验证集
    fold_df = df.iloc[val_idx]
    
    # 输出为CSV文件
    fold_df.to_csv(f'/data3/tongshuo/dataset/image/SICAPv2/ten_fold_balanced/fold_{fold}.csv', index=True)

print("Data has been split into 10 folds and saved as CSV files.")


Data has been split into 10 folds and saved as CSV files.


In [ ]:
import pandas as pd
import numpy as np

# Load your dataset (assuming it's in a CSV file)
df = pd.read_csv("/data3/tongshuo/dataset/image/DR_dataset/ten_fold_balanced/all.csv")

# Step 1: Filter the dataset by diagnosis
diagnosis_0 = df[df['label'] == 0]
diagnosis_1 = df[df['label'] == 1]
diagnosis_2 = df[df['label'] == 2]
diagnosis_3 = df[df['label'] == 3]
diagnosis_4 = df[df['label'] == 4]

# Step 2: Randomly select 600 samples from diagnosis 0
diagnosis_0_selected = diagnosis_0.sample(n=8000, random_state=42)

# Step 3: Select all samples from diagnosis 1, 3, and 4
diagnosis_1_selected = diagnosis_1
diagnosis_3_selected = diagnosis_3
diagnosis_4_selected = diagnosis_4
# Step 4: Randomly select 300 samples from diagnosis 2
diagnosis_2_selected = diagnosis_2.sample(n=4000, random_state=42)

# Step 5: Combine the selected samples
balanced_df = pd.concat([diagnosis_0_selected, diagnosis_1_selected, diagnosis_2_selected, diagnosis_3_selected, diagnosis_4_selected])

# Optionally, shuffle the final dataset if you want random ordering
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# The resulting balanced dataset
print(balanced_df.head())  # Display the first few rows of the balanced dataset
print(len(balanced_df))
# Save the balanced dataset to a new CSV file
balanced_df.to_csv("/data3/tongshuo/dataset/image/DR_dataset/ten_fold_balanced/balanced_all.csv", index=False)


          name  label
0   37638_left      0
1   30483_left      0
2  32809_right      0
3   10701_left      0
4   7817_right      0
16024


In [7]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold

# 读取原始数据
df = pd.read_csv('/data3/tongshuo/dataset/image/DR_dataset/ten_fold_balanced/balanced_all.csv')
# 确保目标列是 'diagnosis'
target_col = 'label'

# 初始化 StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# 保存每个折的数据
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df[target_col])):
    # 分别获取训练集和验证集
    fold_df = df.iloc[val_idx]
    
    # 输出为CSV文件
    fold_df.to_csv(f'/data3/tongshuo/dataset/image/DR_dataset/ten_fold_balanced/fold_{fold}.csv', index=True)

print("Data has been split into 10 folds and saved as CSV files.")


Data has been split into 10 folds and saved as CSV files.


In [3]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold

# 读取原始数据
df = pd.read_csv("/data4/tongshuo/Ordinal_Regression/dataset/Adience/age_merged_data.csv")
# 确保目标列是 'diagnosis'
target_col = 'label'

# 初始化 StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# 保存每个折的数据
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df[target_col])):
    # 分别获取训练集和验证集
    fold_df = df.iloc[val_idx]
    
    # 输出为CSV文件
    fold_df.to_csv(f'/data4/tongshuo/Ordinal_Regression/dataset/Adience/fold_{fold}.csv', index=True)

print("Data has been split into 10 folds and saved as CSV files.")


Data has been split into 10 folds and saved as CSV files.
